In [1]:
from redback import filters
import redback
import bilby
from redback.model_library import all_models_dict
from redback.likelihoods import GaussianLikelihoodQuadratureNoise

import astropy.units as u
import sncosmo

from astroquery.svo_fps import SvoFps

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

%matplotlib widget

/Users/zgl12/anaconda3/lib/python3.11/site-packages/lalsimulation/lalsimulation.py:8: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal
16:23 bilby INFO    : Running bilby version: 2.3.0
16:23 redback INFO    : Running redback version: 1.0.31


In [2]:
df = pd.read_csv('/Users/zgl12/Downloads/mosfit_collated_vxm_data_with_tess.csv')
df['band'] = np.nan

filterss = df['Svo_filter'].unique()

filter_table_master = SvoFps.get_filter_index(500*u.Angstrom, 12000*u.Angstrom, timeout = 400).to_pandas()

filter_table = filter_table_master[filter_table_master['filterID'].isin(filterss)]

filter_table = filter_table_master[filter_table_master['filterID'].isin(filterss)]

filter_table = filter_table.reset_index(drop=True)

df

,MJD,Observatory,Filter,Svo_filter,Flux,d_Flux,Mag,d_Mag,WavelengthEff,FrequencyEff,band
0,58804.625973,TESS,R,TESS/TESS.Red,36.028258,10.395570,20.008392,0.312586,7452.640000,4.022634e+14,NaN
1,58804.875970,TESS,R,TESS/TESS.Red,53.605714,12.955742,19.576972,0.261581,7452.640000,4.022634e+14,NaN
2,58805.125966,TESS,R,TESS/TESS.Red,60.365240,12.956633,19.448033,0.232109,7452.640000,4.022634e+14,NaN
3,58805.375963,TESS,R,TESS/TESS.Red,79.238997,10.134140,19.152653,0.137291,7452.640000,4.022634e+14,NaN
4,58805.625959,TESS,R,TESS/TESS.Red,109.314425,13.228708,18.803306,0.129733,7452.640000,4.022634e+14,NaN
...,...,...,...,...,...,...,...,...,...,...,...
229,58961.960000,Konkoly,V,Generic/Johnson.V,1507.578725,50.575448,15.954300,0.036424,5467.573997,5.483098e+14,NaN
230,58961.960000,Konkoly,g,SLOAN/SDSS.g,1255.220432,36.293471,16.153200,0.031393,4671.782214,6.417090e+14,NaN
231,58961.960000,Konkoly,r,SLOAN/SDSS.r,2333.672992,55.612301,15.479900,0.025874,6141.123004,4.881720e+14,NaN
232,58961.960000,Konkoly,i,SLOAN/SDSS.i,2219.422579,61.142982,15.534400,0.029911,7457.889036,4.019803e+14,NaN


In [3]:

filter_label = [f"{x.split('/')[0]}::" + x.split('/')[1].split('.')[1] for x in filter_table['filterID']]

plot_label   = [x.split('_')[0] for x in filter_table['filterID']]

_ = [filters.add_filter_svo(filter_table.iloc[ii], filter_label[ii], plot_label[ii]) for ii in range(len(filter_table))]

In [4]:
filt_list_rb = filters.show_all_filters().to_pandas()

for ii in range(len(df)):
    
    df.loc[ii, 'band'] = filt_list_rb['bands'][filt_list_rb['label'] == df.loc[ii, 'Svo_filter'].split('_')[0]].reset_index(drop=True)[0]

In [5]:
sampler = "dynesty"
nlive = 3000
walks = 500


df = df[(df['Mag'] > 0.) & (df['Mag'] < 21.)]
# df = df[(df['MJD'] >= 58804.08) & (df['MJD'] <= (58804.08 + 500))]
# df = df[(df['Observatory'] != 'Swift') & (df['Observatory'] != 'PS1')].reset_index(drop=True)
df.sort_values(by='MJD', inplace=True)
time_d = df['MJD'].values # MJD days
mag = df['Mag'].values 
mag_err = df['d_Mag'].values
# frequency = df['FrequencyEff'].values # Effective frequency in Hz of the filter
band = df['band'].values

len(df), len(time_d), len(mag), len(mag_err), len(band)


(234, 234, 234, 234, 234)

In [6]:
sn=redback.transient.Supernova(name            = 'SN2019vxm',
                               data_mode       = 'magnitude',
                               time_mjd        = time_d,
                               magnitude       = mag,
                               magnitude_err   = mag_err,
                               bands           = band,
                               use_phase_model = True
                              )


16:23 redback WARNING : [Errno 2] No such file or directory: 'supernova//SN2019vxm_metadata.csv'
16:23 redback WARNING : Setting metadata to None. This is not an error, but a warning that no metadata could be found online.


In [7]:
model = 't0_supernova_extinction'
base_model = 'csm_interaction' # Physical model

model_kwargs = dict(
                    bands         = sn.filtered_sncosmo_bands, 
                    base_model    = base_model,
                    output_format = 'magnitude'
                    )

function = all_models_dict[model]

In [10]:
from redback.model_library import model_bounds_dict
print(model_bounds_dict['csm_interaction'])

ImportError: cannot import name 'model_bounds_dict' from 'redback.model_library' (/Users/zgl12/anaconda3/lib/python3.11/site-packages/redback/model_library.py)

In [ ]:
priors = redback.priors.get_priors(model=model)
priors.update(redback.priors.get_priors(model=base_model))
priors['redshift'] = 0.019

priors['t0'] = 58804.03
priors['kappa'] = 0.34

# priors['t0'] = bilby.core.prior.Uniform(   
#                                         minimum     = 58804.03 - 0.3,
#                                         maximum     = 58804.03 + 0.3,
#                                         name        = 't0',
#                                         latex_label = r'$t_{\rm expl.}^{}~\rm (day)$',
#                                         unit        = None,
#                                         boundary    = None
#                                         )

priors['mej'] = bilby.core.prior.Uniform(   
                                        minimum     = 1,
                                        maximum     = 400,
                                        name        = 'mej',
                                        latex_label = r'$M_{\rm{ej}}^{}~(M_{\odot})$',
                                        unit        = None,
                                        boundary    = None
                                        )

priors['csm_mass'] = bilby.core.prior.Uniform(   
                                        minimum     = 0.05,
                                        maximum     = 260,
                                        name        = 'csm_mass',
                                        latex_label = r'$M_{\rm{csm}}^{}~(M_{\odot})$',
                                        unit        = None,
                                        boundary    = None
                                        )

priors['vej'] = bilby.core.prior.Uniform(   
                                        minimum     = 1000,
                                        maximum     = 3e5,
                                        name        = 'vej',
                                        latex_label = r'$v_{\rm{ej}}^{}~({\rm km\,s^{-1}})$',
                                        unit        = None,
                                        boundary    = None
                                        )

priors['av'] = bilby.core.prior.Uniform(
                                        minimum     = 0,
                                        maximum     = 1,
                                        name        = 'av',
                                        latex_label = r'$A_V~\rm (mag)$',
                                        unit        = None,
                                        boundary    = None
                                        )

priors['eta'] = bilby.core.prior.Uniform(minimum = -100, maximum = 100, name = 'csm density profile exponent', 
                                         latex_label = r'$\eta$', unit = None,
                                         boundary = None)

priors['r0'] = bilby.core.prior.Uniform(minimum = 0, maximum = 10000.0, name = 'r0', 
                                        latex_label = r'$r_{0}~\rm (Au)$', unit = None,
                                        boundary = None)

priors['rho'] = bilby.core.prior.Uniform(minimum = 0, maximum = 1100, name = 'csm density profile amplitude', 
                                         latex_label = r'$\rho$', unit = None,
                                         boundary = None)

priors['temperature_floor'] = bilby.core.prior.Uniform(
                                        minimum     = 100,            
                                        maximum     = 20000,        
                                        name        = 'temperature_floor', 
                                        latex_label = r'$T~(\rm K)$',                     
                                        unit        = None,
                                        boundary    = None
                                        )

likelihood_func = GaussianLikelihoodQuadratureNoise(   
                                        x        = sn.x,
                                        y        = sn.y,
                                        function = function,
                                        kwargs   = model_kwargs,
                                        sigma_i  = sn.y_err
                                        )


16:23 redback WARNING : [Errno 2] No such file or directory: '/Users/zgl12/anaconda3/lib/python3.11/site-packages/redback/priors/t0_supernova_extinction.prior'
16:23 redback WARNING : Returning empty PriorDict.


In [9]:
result = redback.fit_model(transient    = sn,
                           model        = model,
                           sampler      = sampler,
                           model_kwargs = model_kwargs,
                           prior        = priors,
                           sample       = 'rslice',
                           nlive        = nlive,
                           resume       = True,
                           likelihood   = likelihood_func,
                           clean        = True,
                           walks        = walks,
                           plot         = False
                          )

16:23 bilby INFO    : Running for label 'SN2019vxm', output will be saved to 'supernova//t0_supernova_extinction'
16:24 bilby INFO    : Analysis priors:
16:24 bilby INFO    : mej=Uniform(minimum=1, maximum=400, name='mej', latex_label='$M_{\\rm{ej}}^{}~(M_{\\odot})$', unit=None, boundary=None)
16:24 bilby INFO    : csm_mass=Uniform(minimum=0.05, maximum=260, name='csm_mass', latex_label='$M_{\\rm{csm}}^{}~(M_{\\odot})$', unit=None, boundary=None)
16:24 bilby INFO    : vej=Uniform(minimum=1000, maximum=300000.0, name='vej', latex_label='$v_{\\rm{ej}}^{}~({\\rm km\\,s^{-1}})$', unit=None, boundary=None)
16:24 bilby INFO    : eta=Uniform(minimum=-100, maximum=100, name='csm density profile exponent', latex_label='$\\eta$', unit=None, boundary=None)
16:24 bilby INFO    : rho=Uniform(minimum=0, maximum=1100, name='csm density profile amplitude', latex_label='$\\rho$', unit=None, boundary=None)
16:24 bilby INFO    : r0=Uniform(minimum=0, maximum=10000.0, name='r0', latex_label='$r_{0}~\\rm (

ValueError: One of the requested xi is out of bounds in dimension 1

In [ ]:
result.plot_corner(show_titles=True, title_quantiles = [0.16, 0.5, 0.84])

In [ ]:
cols = result.posterior.columns.values

for col in cols:
    arr = result.posterior[col].values
    print(f"{col}")
    print(f"${np.nanmedian(arr):.2f}" + "^{" + f"{abs(np.nanpercentile(arr, 84) - np.nanmedian(arr)):.2f}" + "}" + "_{" + f"{abs(np.nanmedian(arr) - np.nanpercentile(arr, 16)):.2f}" + "}$")

In [ ]:
print(result.log_evidence)
print(result.log_evidence_err)
print(result.bayesian_model_dimensionality)
print(result.covariance_matrix)

In [ ]:
mej = result.posterior['mej'].values

print(np.nanmedian(mej), np.nanmedian(mej) - np.nanpercentile(mej, 16), np.nanpercentile(mej, 84) - np.nanmedian(mej))

plt.figure(figsize=(10, 5))
plt.title('mej')
plt.hist(mej, bins=50)
plt.xlabel('mej')
plt.ylabel('Counts')
plt.axvline(np.nanmedian(mej), color='red', linestyle='--')
plt.show()

In [ ]:
from redback import filters
import redback
import bilby
from redback.model_library import all_models_dict
from redback.likelihoods import GaussianLikelihoodQuadratureNoise

import astropy.units as u
import sncosmo

from astroquery.svo_fps import SvoFps

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

%matplotlib widget

df = pd.read_csv('/Users/zgl12/Downloads/mosfit_collated_vxm_data_with_tess.csv')
df['band'] = np.nan

filterss = df['Svo_filter'].unique()

filter_table_master = SvoFps.get_filter_index(500*u.Angstrom, 12000*u.Angstrom, timeout = 400).to_pandas()

filter_table = filter_table_master[filter_table_master['filterID'].isin(filterss)]
# filter_table = filter_table_master[filter_table_master['filterID'].isin(filterss)]
filter_table = filter_table.reset_index(drop=True)

filter_label = [f"{x.split('/')[0]}::" + x.split('/')[1].split('.')[1] for x in filter_table['filterID']]
plot_label   = [x.split('_')[0] for x in filter_table['filterID']]

_ = [filters.add_filter_svo(filter_table.iloc[ii], filter_label[ii], plot_label[ii]) for ii in range(len(filter_table))]

filt_list_rb = filters.show_all_filters().to_pandas()

for ii in range(len(df)):
    
    df.loc[ii, 'band'] = filt_list_rb['bands'][filt_list_rb['label'] == df.loc[ii, 'Svo_filter'].split('_')[0]].reset_index(drop=True)[0]

sampler = "dynesty"
nlive = 3000
walks = 500


df = df[(df['Mag'] > 0.) & (df['Mag'] < 21.)]
# df = df[(df['MJD'] >= 58804.08) & (df['MJD'] <= (58804.08 + 500))]
# df = df[(df['Observatory'] != 'Swift') & (df['Observatory'] != 'PS1')].reset_index(drop=True)
df.sort_values(by='MJD', inplace=True)
time_d = df['MJD'].values # MJD days
mag = df['Mag'].values 
mag_err = df['d_Mag'].values
# frequency = df['FrequencyEff'].values # Effective frequency in Hz of the filter
band = df['band'].values

sn=redback.transient.Supernova(name            = 'SN2019vxm',
                               data_mode       = 'magnitude',
                               time_mjd        = time_d,
                               magnitude       = mag,
                               magnitude_err   = mag_err,
                               bands           = band,
                               use_phase_model = True
                              )

model = 't0_supernova_extinction'
base_model = 'csm_interaction' # Physical model

model_kwargs = dict(
                    bands         = sn.filtered_sncosmo_bands, 
                    base_model    = base_model,
                    output_format = 'magnitude'
                    )

function = all_models_dict[model]

priors = redback.priors.get_priors(model=model)
priors.update(redback.priors.get_priors(model=base_model))
priors['redshift'] = 0.019

priors['t0'] = 58804.03
priors['kappa'] = 0.34

# priors['t0'] = bilby.core.prior.Uniform(   
#                                         minimum     = 58804.08 - 0.36,
#                                         maximum     = 58804.08 + 0.36,
#                                         name        = 't0',
#                                         latex_label = r'$t_{\rm expl.}^{}~\rm (day)$',
#                                         unit        = None,
#                                         boundary    = None
#                                         )

priors['mej'] = bilby.core.prior.Uniform(   
                                        minimum     = 1,
                                        maximum     = 400,
                                        name        = 'mej',
                                        latex_label = r'$M_{\rm{ej}}^{}~(M_{\odot})$',
                                        unit        = None,
                                        boundary    = None
                                        )

priors['csm_mass'] = bilby.core.prior.Uniform(   
                                        minimum     = 0.05,
                                        maximum     = 260,
                                        name        = 'csm_mass',
                                        latex_label = r'$M_{\rm{csm}}^{}~(M_{\odot})$',
                                        unit        = None,
                                        boundary    = None
                                        )

priors['vej'] = bilby.core.prior.Uniform(   
                                        minimum     = 1000,
                                        maximum     = 3e5,
                                        name        = 'vej',
                                        latex_label = r'$v_{\rm{ej}}^{}~({\rm km\,s^{-1}})$',
                                        unit        = None,
                                        boundary    = None
                                        )

priors['av'] = bilby.core.prior.Uniform(
                                        minimum     = 0,
                                        maximum     = 1,
                                        name        = 'av',
                                        latex_label = r'$A_V~\rm (mag)$',
                                        unit        = None,
                                        boundary    = None
                                        )

priors['eta'] = bilby.core.prior.Uniform(minimum = -100, maximum = 100, name = 'csm density profile exponent', 
                                         latex_label = r'$\eta$', unit = None,
                                         boundary = None)

priors['r0'] = bilby.core.prior.Uniform(minimum = 0, maximum = 10000.0, name = 'r0', 
                                        latex_label = r'$r_{0}~\rm (Au)$', unit = None,
                                        boundary = None)

priors['rho'] = bilby.core.prior.Uniform(minimum = 0, maximum = 1100, name = 'csm density profile amplitude', 
                                         latex_label = r'$\rho$', unit = None,
                                         boundary = None)

priors['temperature_floor'] = bilby.core.prior.Uniform(
                                        minimum     = 100,            
                                        maximum     = 20000,        
                                        name        = 'temperature_floor', 
                                        latex_label = r'$T~(\rm K)$',                     
                                        unit        = None,
                                        boundary    = None
                                        )

likelihood_func = GaussianLikelihoodQuadratureNoise(   
                                        x        = sn.x,
                                        y        = sn.y,
                                        function = function,
                                        kwargs   = model_kwargs,
                                        sigma_i  = sn.y_err
                                        )

result = redback.fit_model(transient    = sn,
                           model        = model,
                           sampler      = sampler,
                           model_kwargs = model_kwargs,
                           prior        = priors,
                           sample       = 'rslice',
                           nlive        = nlive,
                           resume       = True,
                           likelihood   = likelihood_func,
                           clean        = True,
                           walks        = walks,
                           plot         = False
                          )